In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import re
import numpy as np
import glob
import os.path
import mne
import resampy
# !pip install librosa
import librosa
from scipy.signal import resample, hann
#!pip install PyWavelets
import pywt
from sklearn import preprocessing
from scipy import signal
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
log = logging.getLogger(__name__)

## given categories

In [ ]:
data_folders = ['clips/www.isip.piconepress.com/normal/tuh_eeg/downloads/tuh_eeg_seizure/v1.5.2/edf/train/01_tcp_ar/']

In [ ]:
def session_key(file_name):
    """ sort the file name by session """
    return re.findall(r'(s\d{2})', file_name)


def natural_key(file_name):
    """ provides a human-like sorting key of a string """
    key = [int(token) if token.isdigit() else None
           for token in re.split(r'(\d+)', file_name)]
    return key

def time_key(file_name):
    """ provides a time-based sorting key """
    splits = file_name.split('/')
    [date] = re.findall(r'(\d{4}_\d{2}_\d{2})', splits[-2])
    date_id = [int(token) for token in date.split('_')]
    recording_id = natural_key(splits[-1])
    session_id = session_key(splits[-2])

    return date_id + session_id + recording_id


def read_all_file_names(path, extension, key="time"):
    """ read all files with specified extension from given path
    :param path: parent directory holding the files directly or in subdirectories
    :param extension: the type of the file, e.g. '.txt' or '.edf'
    :param key: the sorting of the files. natural e.g. 1, 2, 12, 21 (machine 1, 12, 2, 21) or by time since this is
    important for cv. time is specified in the edf file names
    """
    file_paths = glob.glob(path + '**/*' + extension, recursive=True)

    if key == 'time':
        return sorted(file_paths, key=time_key)

    elif key == 'natural':
        return sorted(file_paths, key=natural_key)
    
def get_all_sorted_file_names_and_labels_by_channel(full_folder):
    all_file_names = []
    for folder in full_folder:
        full_folder = os.path.join(folder) + '/'
        #log.info("Reading {:s}...".format(full_folder))
        this_file_names = read_all_file_names(full_folder, '.edf', key='time')
        #log.info(".. {:d} files.".format(len(this_file_names)))
        all_file_names.extend(this_file_names)
    #log.info("{:d} files in total.".format(len(all_file_names)))
    all_file_names = sorted(all_file_names, key=time_key)

    labels = ['/abnormal/' in f for f in all_file_names]
    labels = np.array(labels).astype(np.int64)
    return all_file_names, labels


In [ ]:
all_file_names, labels = get_all_sorted_file_names_and_labels_by_channel(data_folders)

## given full link

In [ ]:
all_file = []

with open('file.txt', 'r') as src:  
     for line in src:
            line = line.replace('https://', '/Users/').rstrip('\n')
            if line not in all_file:
                all_file.append(line)

In [ ]:
data_folders = all_file

In [ ]:
def get_all_sorted_file_names_and_labels_by_type(full_folder):
    all_file_names = []
    for folder in full_folder:
        this_file_names = os.path.join(folder)
        all_file_names.append(this_file_names)
    all_file_names = sorted(all_file_names)

    labels = ['/tuh_eeg/' in f for f in all_file_names] 
    labels = np.array(labels).astype(np.int64)
    return all_file_names, labels


In [ ]:
all_file_names, labels = get_all_sorted_file_names_and_labels_by_type(data_folders)

## pre-process

In [ ]:
max_recording_mins = 35  # exclude larger recordings from training set
n_recordings = None  # set to an integer, if you want to restrict the set size
sensor_types = ["EEG"]
sec_to_cut = 60  # cut away at start of each recording
duration_recording_mins = 20  # how many minutes to use per recording
max_abs_val = 800  # for clipping
sampling_freq = 256 # resample
divisor = 10  # divide signal by this
max_hz = 256

In [ ]:
def get_recording_length(file_path):
    """ some recordings were that huge that simply opening them with mne caused the program to crash. therefore, open
    the edf as bytes and only read the header. parse the duration from there and check if the file can safely be opened
    :param file_path: path of the directory
    :return: the duration of the recording
    """
    f = open(file_path, 'rb')
    header = f.read(256)
    f.close()

    return int(header[236:244].decode('ascii'))



def get_recording_detail(file_path):
    """ # renew 
    :param file_path: path of the directory
    :return: the duration of the recording, birth year of patient, gender of patient
    """
    f = open(file_path, 'rb')
    header = f.read(256)
    length = int(header[236:244].decode('ascii'))
    year = int(header[26:30].decode('ascii'))
    gender = str(header[17:18].decode('ascii'))
    f.close()

    return length, year, gender

In [ ]:
if max_recording_mins is not None:
    lengths = [get_recording_length(fname) for fname in all_file_names]
    lengths = np.array(lengths)
    mask = lengths < max_recording_mins * 60
    cleaned_file_names = np.array(all_file_names)[mask]
    cleaned_labels = labels[mask]
else:
    cleaned_file_names = np.array(all_file_names)
    cleaned_labels = labels

In [ ]:
preproc_functions = []

preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(sec_to_cut * fs)], fs))

In [ ]:
def get_info_with_mne(file_path):
    """ read info from the edf file without loading the data. loading data is done in multiprocessing since it takes
    some time. getting info is done before because some files had corrupted headers or weird sampling frequencies
    that caused the multiprocessing workers to crash. therefore get and check e.g. sampling frequency and duration
    beforehand
    :param file_path: path of the recording file
    :return: file name, sampling frequency, number of samples, number of signals, signal names, duration of the rec
    """
    try:
        edf_file = mne.io.read_raw_edf(file_path, verbose='error')
    except ValueError:
        return None, None, None, None, None, None
    sampling_frequency = int(edf_file.info['sfreq'])
    if sampling_frequency < 10:
        sampling_frequency = 1 / (edf_file.times[1] - edf_file.times[0])
        if sampling_frequency < 10:
            return None, sampling_frequency, None, None, None, None

    n_samples = edf_file.n_times
    signal_names = edf_file.ch_names
    n_signals = len(signal_names)
    duration = n_samples / max(sampling_frequency, 1)

    return edf_file, sampling_frequency, n_samples, n_signals, signal_names, duration

In [ ]:
def load_data(fname, preproc_functions):
    cnt, sfreq, n_samples, n_channels, chan_names, n_sec = get_info_with_mne(
        fname)
    log.info("Load data...")
    cnt.load_data()
    selected_ch_names = []
    wanted_elecs = ['C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1','FP2', 'FZ', 'O1', 'O2',
                    'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6'] #'A1', 'A2' --> not in 03
 
    for wanted_part in wanted_elecs:
        wanted_found_name = []
        for ch_name in cnt.ch_names:
            if ' ' + wanted_part + '-' in ch_name:
                wanted_found_name.append(ch_name)
        assert len(wanted_found_name) == 1
        selected_ch_names.append(wanted_found_name[0])

    cnt = cnt.pick_channels(selected_ch_names)

    assert np.array_equal(sorted(cnt.ch_names), sorted(selected_ch_names))
    
    n_sensors = 0
    n_sensors += 19

    assert len(cnt.ch_names)  == n_sensors, (
        "Expected {:d} channel names, got {:d} channel names".format(
            n_sensors, len(cnt.ch_names)))

    # change from volt to mikrovolt
    data = (cnt.get_data() * 1e6).astype(np.float32)
    fs = cnt.info['sfreq']
    log.info("Preprocessing...")
    for fn in preproc_functions:
        log.info(fn)
        data, fs = fn(data, fs)
        data = data.astype(np.float32)
        fs = float(fs)
    return data

In [ ]:
X = []
y = []
n_files = len(cleaned_file_names[:n_recordings])
for i_fname, fname in enumerate(cleaned_file_names[:n_recordings]):
    log.info("Load {:d} of {:d}".format(i_fname + 1,n_files))
    x = load_data(fname, preproc_functions=preproc_functions)
    assert x is not None
    X.append(x)
    y.append(cleaned_labels[i_fname])
y = np.array(y)

In [ ]:
data = X[0]
for i in range(1, len(X)):
    data = np.append(data, X[i], axis=1)

In [ ]:
def FFT(data):
    """
    Apply Fast Fourier Transform to the last axis.
    """
    axis = data.ndim - 1
    return np.fft.rfft(data, axis=axis)


In [ ]:
def Log10(data):
    """
    Apply Log10
    """
    # 10.0 * log10(re * re + im * im)
    indices = np.where(data <= 0)
    data[indices] = np.max(data)
    data[indices] = (np.min(data) * 0.1)
    return np.log10(data)

In [ ]:
def Magnitude(data):
    return np.absolute(data)

In [ ]:
def MagnitudeAndPhase(data):
    """
    Take the magnitudes and phases of complex data and append them together.
    """
    magnitudes = np.absolute(data)
    phases = np.angle(data)
    return np.concatenate((magnitudes, phases), axis=1)

In [ ]:
def stats_channel(data):
    """
    Take (mean,standard_deviation, min, max) for each channel.
    """
    # data[ch][dim]
    shape = data.shape
    out = np.empty((shape[0], 4))
    for i in range(len(data)):
        ch_data = data[i]
        outi = out[i]
        outi[0] = np.mean(ch_data)
        outi[1] = np.std(ch_data)
        outi[2] = np.min(ch_data)
        outi[3] = np.max(ch_data)

    return out

In [ ]:
def stats(data):
    """
    Take (mean,standard_deviation, min, max) for data.
    """
    # data[ch][dim]
    
    out = []
    out.append(np.mean(data))
    out.append(np.std(data))
    out.append(np.min(data))
    out.append(np.max(data))


    return out

In [ ]:
def mfcc(data):
    """
    Mel-frequency cepstrum coefficients
    """
    all_mfccs = []
    for ch in data:
        mfccs = librosa.feature.mfcc(y=ch,n_mfcc = 13, sr=sampling_freq)[2:13] 
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfccs = librosa.feature.delta(mfccs,order=2)
        comprehensice_mfccs = np.concatenate((mfccs, delta_mfccs, delta2_mfccs),axis = 0)
        all_mfccs.append(comprehensice_mfccs.ravel())

    return np.array(all_mfccs)


In [ ]:
def Resample(data):
    """
    Resample time-series data.
    """
    axis = data.ndim - 1
    if data.shape[-1] > sampling_freq:
        return resample(data, sampling_freq, axis=axis)
    return data

In [ ]:
def ResampleHanning(data):
    """
    Resample time-series data using a Hanning window
    """
    axis = data.ndim - 1
    out = resample(data, sampling_freq, axis=axis, window=hann(M=data.shape[axis]))
    return out

In [ ]:
def UnitScale(data):
    """
    Scale across the last axis.
    """
    return preprocessing.scale(data, axis=data.ndim-1)

In [ ]:
def UnitScaleFeat(data):
    """
    Scale across the first axis, i.e. scale each feature.
    """
    return preprocessing.scale(data, axis=0)

In [ ]:
def CorrelationMatrix(data):
    """
    Calculate correlation coefficients matrix across all EEG channels.
    """
    return np.corrcoef(data)

In [ ]:
def Eigenvalues(data):
    """
    Take eigenvalues of a matrix, and sort them by magnitude in order to
    make them useful as features (as they have no inherent order).
    """
    w, v = np.linalg.eig(data)
    w = np.absolute(w)
    w.sort()
    return w

In [ ]:
# Take the upper right triangle of a matrix
def upper_right_triangle(matrix):
    accum = []
    for i in range(matrix.shape[0]):
        for j in range(i+1, matrix.shape[1]):
            accum.append(matrix[i, j])

    return np.array(accum)

In [ ]:
def Freq_Correlation(data):
    
        """
        Correlation in the frequency domain. First take FFT with (start, end) slice options,
        then calculate correlation co-efficients on the FFT output, followed by calculating
        eigenvalues on the correlation co-efficients matrix.
        The output features are ( upper_right_diagonal(correlation_coefficients)
        Features can be selected/omitted using the constructor arguments.
        """
        data1 = FFT(data)
        data1 = Magnitude(data1)
        data1 = Log10(data1)

        data2 = data1

        data2 = UnitScaleFeat(data2)
        
        data2 = CorrelationMatrix(data2)

        w = Eigenvalues(data2)

        out = []
        data2 = upper_right_triangle(data2)
        out.append(data2)

        for d in out:
            assert d.ndim == 1

        return np.concatenate(out, axis=0)


In [ ]:
def Time_Correlation(data):
    """
    Correlation in the time domain. First downsample the data, then calculate correlation co-efficients
    followed by calculating eigenvalues on the correlation co-efficients matrix.
    The output features are (upper_right_diagonal(correlation_coefficients), eigenvalues)
    Features can be selected/omitted using the constructor arguments.
    """

    # so that correlation matrix calculation doesn't crash
    for ch in data:
        if np.alltrue(ch == 0.0):
            ch[-1] += 0.00001

    data1 = data
    if data1.shape[1] > max_hz: 
        data1 = Resample(data1)

    data1 = UnitScaleFeat(data1)

    data1 = CorrelationMatrix(data1)

    out = []
    data1 = upper_right_triangle(data1)
    out.append(data1)

    for d in out:
        assert d.ndim == 1

    return np.concatenate(out, axis=0)

In [ ]:
def TimeFreqCorrelation(data):
    """
    Combines time and frequency correlation, taking both correlation coefficients and eigenvalues.
    """
    data1 = TimeCorrelation(data)
    data2 = FreqCorrelation(data)
    assert data1.ndim == data2.ndim
    return np.concatenate((data1, data2), axis=data1.ndim-1)

In [ ]:

f=open('./descriptive/stats_TCSZ.txt','wb')
f.write(b"stats of data \n")
np.savetxt(f,stats_all,newline=", ")
f.write(b"\n\n")
f.write(b"stats of mfccs \n")
np.savetxt(f,stats_mfccs, newline=", ")
f.write(b"\n\n")
f.write(b"stats of Freq Correlation \n")
np.savetxt(f, stats(FreqCorrelation), newline=", ")
f.write(b"\n\n")
f.write(b"stats of Time Correlation \n")
np.savetxt(f, stats(TimeCorrelation), newline=", ")
f.write(b"\n\n")
f.write(b"Eigenvalues \n")
np.savetxt(f, eigenvalues, newline=", ")
f.write(b"\n\n")
f.write(b"stats of channel \n")
np.savetxt(f,stats_channels, newline=", ")
f.write(b"\n\n")
f.write(b"FreqCorrelation \n")
np.savetxt(f,FreqCorrelation, newline=", ")
f.write(b"\n\n")
f.write(b"Time Correlation \n")
np.savetxt(f,TimeCorrelation, newline=", ")

f.close()

f.close()